In [1]:
import os
import numpy as np
import pickle

In [ ]:
def load_cifar10_batch(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
    images = dict[b'data'].reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = dict[b'labels']
    return images, labels

# Path to the CIFAR-10 batches
path_to_batches = 'cifar-10'  # Adjust the path according to your directory structure

# Loading the training data
train_images = []
train_labels = []
for i in range(1, 6):  # CIFAR-10 has 5 training batches
    file = os.path.join(path_to_batches, f'data_batch_{i}')
    images, labels = load_cifar10_batch(file)
    train_images.append(images)
    train_labels.append(labels)

train_images = np.concatenate(train_images, axis=0)
train_labels = np.concatenate(train_labels, axis=0)

# Loading the test data
test_file = os.path.join(path_to_batches, 'test_batch')
test_images, test_labels = load_cifar10_batch(test_file)

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert labels to one-hot encoding
train_labels = np.eye(10)[train_labels]
test_labels = np.eye(10)[test_labels]

# Check the shapes
print("Training data shape:", train_images.shape)   # Example: (50000, 32, 32, 3)
print("Training labels shape:", train_labels.shape) # Example: (50000, 10)
print("Test data shape:", test_images.shape)        # Example: (10000, 32, 32, 3)
print("Test labels shape:", test_labels.shape)      # Example: (10000, 10)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Building the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# Adding fully connected layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))  # 10 classes

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
# model.summary()

In [5]:
# Set training parameters
epochs = 20  # Can be adjusted
batch_size = 64  # Can be adjusted

In [ ]:
# Train the model
history = model.fit(train_images, train_labels, epochs=epochs, 
                    batch_size=batch_size, 
                    validation_split=0.2)  # using 20% of the training data for validation

# Save the model (optional)
# model.save('cifar10_model.h5')

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [8]:
import matplotlib.pyplot as plt

In [ ]:
# Extracting accuracy and loss values
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

# Plotting training and validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [10]:
from tensorflow.keras.layers import Dropout

In [11]:
# Adjusting the CNN model to include dropout
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.15))  # Dropout layer after pooling

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.15))  # Another dropout layer

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))  # Dropout layer before the final dense layer
model.add(Dense(10, activation='softmax'))

# Recompile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Retrain the model
history = model.fit(train_images, train_labels, epochs=epochs, 
                    batch_size=batch_size, 
                    validation_split=0.2)

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
# Extracting accuracy and loss values
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

# Plotting training and validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Assuming that you have a model named `model` and a test set `test_images` and `test_labels`

# Get predictions from the model
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)

# Compute confusion matrix
conf_mtx = confusion_matrix(true_classes, predicted_classes)

# Compute precision, recall, and F1 score
class_report = classification_report(true_classes, predicted_classes)

print("Confusion Matrix:\n", conf_mtx)
print("\nClassification Report:\n", class_report)


In [16]:
import seaborn as sns

In [ ]:
report_dict = classification_report(true_classes, predicted_classes, output_dict=True)

# Exclude non-class keys such as 'accuracy' and 'macro avg', which do not have 'precision', 'recall', 'f1-score' subkeys
class_labels = [key for key in report_dict.keys() if key not in ('accuracy', 'macro avg', 'weighted avg')]

# Now use list comprehensions to get the metrics for each class label
precision_scores = [report_dict[label]['precision'] for label in class_labels]
recall_scores = [report_dict[label]['recall'] for label in class_labels]
f1_scores = [report_dict[label]['f1-score'] for label in class_labels]

# Plotting the graphs
fig, ax = plt.subplots(3, 1, figsize=(10, 15), sharex=True)

sns.barplot(x=class_labels, y=precision_scores, ax=ax[0], palette="viridis")
ax[0].set_title('Precision per Class')
ax[0].set_ylabel('Precision')

sns.barplot(x=class_labels, y=recall_scores, ax=ax[1], palette="viridis")
ax[1].set_title('Recall per Class')
ax[1].set_ylabel('Recall')

sns.barplot(x=class_labels, y=f1_scores, ax=ax[2], palette="viridis")
ax[2].set_title('F1 Score per Class')
ax[2].set_ylabel('F1 Score')
ax[2].set_xlabel('Classes')

plt.xticks(rotation=90)
plt.tight_layout()  # Adjust the layout
plt.show()
